<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing a LangChain + Google PaLM Application</h1>

LLM orchestration frameworks such as LangChain provide abstractions that enable users to build powerful applications in a few lines of code. However, the same abstractions make it difficult to understand what is going on under the hood and to pinpoint the cause of issues.

Phoenix makes your LLM applications *observable* by providing tracing.

In this tutorial, you will:
- Build a simple LLM application using LangChain and Google PaLM,
- Record trace data in OpenInference format,
- Inspect the traces and spans of your application to identify uncaught exceptions and sources of latency and cost.

⚠️ This notebook requires access to the [Google PaLM API](https://developers.generativeai.google/) and a Google API key.

❗ Phoenix tracing is in alpha. 

## 1. Install Dependencies and Import Libraries

Install Phoenix and LangChain.

In [ ]:
!pip install -qq "arize-phoenix[experimental]==0.0.33rc2" langchain

Import libraries.

In [ ]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import numpy as np
import pandas as pd
import phoenix as px
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatGooglePalm
from langchain.embeddings import GooglePalmEmbeddings
from langchain.retrievers import KNNRetriever
from phoenix.experimental.callbacks.langchain_tracer import OpenInferenceTracer
from phoenix.trace.trace_dataset import TraceDataset
from tqdm import tqdm

## 2. Configure Your Google API Key

Set your Google API key.

In [ ]:
google_api_key = os.environ.get("GOOGLE_API_KEY", getpass("Enter your Google API Key: "))
os.environ["GOOGLE_API_KEY"] = google_api_key

## 3. Instantiate Your OpenInference Tracer

Instantiate a tracer to record your trace data in [OpenInference format](https://github.com/Arize-ai/open-inference-spec), an open standard for capturing and storing AI model inferences that enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as Phoenix.

In [ ]:
tracer = OpenInferenceTracer()

## 4. Build Your LLM Application

Define a `RetrievalQA` chain leveraging "embedding-gecko-001" and "chat-bison-001" from Google PaLM. The knowledge base of this chain is built over the Arize documentation.

In [ ]:
embeddings = GooglePalmEmbeddings(model_name="models/embedding-gecko-001")
database_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone-vertexai/database.parquet"
)
knn_retriever = KNNRetriever(
    index=np.stack(database_df["text_vector"]),
    texts=database_df["text"].tolist(),
    embeddings=embeddings,
)
llm = ChatGooglePalm(model_name="models/chat-bison-001")
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=knn_retriever,
)

# 5. Run the Chain

Download a small dataset of user queries to ask your application.

In [ ]:
url = "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries[:10]

Run your chain against 10 queries.

In [ ]:
for query in tqdm(queries[:10]):
    chain.run(query, callbacks=[tracer])

## 6. Launch Phoenix

Launch Phoenix to view your trace and span data.

In [ ]:
spans = tracer.get_spans()
ds = TraceDataset.from_spans(spans)
px.launch_app(trace=ds)